<a href="https://colab.research.google.com/github/josephalanis/thesis_longitudinal_data/blob/main/ThesisRN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
from functools import partial
import math as math
from fractions import Fraction
from sklearn.metrics import DistanceMetric
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances
# Creating code to handle Missing data
from sklearn.metrics.pairwise import nan_euclidean_distances
from sklearn.impute import KNNImputer


#Converting a fraction into a p-adic number
%pip install pyadic
from pyadic import PAdic, ModP

In [ ]:
# Set the path to your dataset and Info.txt file

path ="/content/drive/MyDrive/Colab/Thesis/"
info_file ="/content/drive/MyDrive/Colab/Thesis/full1.csv"
#info_file1="/content/drive/MyDrive/Colab/Thesis/dtTrail3.csv"
#info_file2="/content/drive/MyDrive/Colab/Thesis/s2.csv"


Original=pd.read_csv(info_file)
#sample=pd.read_csv(info_file2)
#info = info.drop('Unnamed: 7', axis=1)
#Miss = pd.read_csv(info_file)


In [ ]:
#Viewing Dataframe

#print(sample.head())
print(Original.head())


   age    tenure  hours  wks_work   ln_wage   ttl_exp
0   20  0.916667     40        51  1.589977  2.256410
1   25  1.500000     32        52  1.778681  3.775641
2   28  1.833333     45        75  2.551715  5.294872
3   33  1.916667     42        97  2.614172  7.160256
4   35  3.916667     45        95  2.536374  8.987180


In [ ]:
#Adding tt1 and tt2
Original['tt1'] = Original.loc[:, 'ttl_exp']
Original['tt2'] = Original.loc[:, 'ttl_exp']
print(Original.head())


   age    tenure  hours  wks_work   ln_wage   ttl_exp       tt1       tt2
0   20  0.916667     40        51  1.589977  2.256410  2.256410  2.256410
1   25  1.500000     32        52  1.778681  3.775641  3.775641  3.775641
2   28  1.833333     45        75  2.551715  5.294872  5.294872  5.294872
3   33  1.916667     42        97  2.614172  7.160256  7.160256  7.160256
4   35  3.916667     45        95  2.536374  8.987180  8.987180  8.987180


In [ ]:
#Creating random sample of 100
sp=Original.sample(n=100)
print(sp.head())

       age     tenure  hours  wks_work   ln_wage    ttl_exp        tt1  \
2943    28   0.166667      6        52  1.607938   2.916667   2.916667   
6656    25   2.333333     40        33  1.741456   3.820513   3.820513   
10198   32   3.416667      6        55  1.682148   4.634615   4.634615   
190     38  10.583333     24        53  2.049209  10.583333  10.583333   
5118    35   2.750000     28        54  1.653161   2.788462   2.788462   

             tt2  
2943    2.916667  
6656    3.820513  
10198   4.634615  
190    10.583333  
5118    2.788462  


In [ ]:
# Code for missing data
import numpy as np


class Corruptor:

    def __init__(self, data, dtype=np.float64):
        self.dtype = data.dtype
        self.shape = np.shape(data)
        self.data = data.astype(dtype)

    def mcar(self):
        """ Overwrites values with MCAR placed NaN's """
        data_1d = self.flatten()
        n_total = len(data_1d)
        nan_x = np.random.choice(range(n_total),
                                  size=int(n_total*0.3),
                                  replace=False)
        for x_i in nan_x:
            data_1d[x_i] = np.nan
        output = data_1d.reshape(self.shape)
        return output


In [ ]:
class Corruptor1:

    def __init__(self, data, dtype=np.float64):
        self.dtype = data.dtype
        self.shape = np.shape(data)
        self.data = data.astype(dtype)

    def mcar1(self):
        """ Overwrites values with MCAR placed NaN's """
        data_1d1 = self.flatten()
        n_total = len(data_1d1)
        nan_x1 = np.random.choice(range(n_total),
                                  size=int(n_total*0.5),
                                  replace=False)

        for x_i1 in nan_x1:
            data_1d1[x_i1] = np.nan
        output1 = data_1d1.reshape(self.shape)
        return output1


In [ ]:
#Creating code to find bias
def bias(X,Y):
    diff=X-Y
    #print(diff)
    m=diff.mean()
    return m

In [ ]:
#Calculating RMSE
from sklearn.metrics import mean_squared_error


In [ ]:
#creating for loop to find everything
imputer=KNNImputer(n_neighbors=3,weights='uniform',metric='nan_euclidean')
nan=float("NaN")

#results=np.zeros((100,4))

def thesis(X):
  results=np.zeros((1,4))
  num_test=results.shape[0]
  num_train=results.shape[1]
  for i in range(num_test):
    #for j in range(4):
      #Creating random sample of 100
      sp=Original.sample(n=1000)
      #removing the columns I don't need and creating missing data:
      sc=np.array([sp['tt1']])
      sc=sc.astype(float)
      sc1=np.array([sp['tt2']])
      sc1=sc1.astype(float)
      m=Corruptor.mcar(sc)
      m=m.flatten()
      m1=Corruptor1.mcar1(sc1)
      m1=m1.flatten()
      #print(m1)
      #Need to combine these into one data frame with BP0,m,m1
      sd=np.array(sp['ttl_exp'])
      sd=sd.astype(float)
      #Combined as an array
      arr = np.column_stack((sd, m,m1))
      #Combined as a dataframe
      imp=pd.DataFrame({'ttl_exp':arr[:,0],'BP1':arr[:,1],'BP2':arr[:,2]})
      #print(imp)
      #Now try the imputer on the data frame of tt1 and tt2
      sp_imputed=np.round(imputer.fit_transform(imp),3)
      #print(sp_imputed.shape)
      sp_imputed=pd.DataFrame({'ttl_exp':sp_imputed[:,0],'tt1':sp_imputed[:,1],'tt2':sp_imputed[:,2]})
      #Find the  bias
      b1=bias(sp_imputed['ttl_exp'],sp_imputed['tt1'])
      b2=bias(sp_imputed['ttl_exp'],sp_imputed['tt2'])
      type(b1)==float
      type(b2)==float
      b1=round(b1,4)
      b2=round(b2,4)
      #Find RMSE
      r1 = mean_squared_error(sp_imputed['ttl_exp'],sp_imputed['tt1'], squared=False)
      type(r1)==float
      r1=round(r1,4)
      r2 = mean_squared_error(sp_imputed['ttl_exp'],sp_imputed['tt2'], squared=False)
      type(r2)==float
      r2=round(r2,4)
      #Storing Results
      results[i,0]=b1
      results[i,1]=b2
      results[i,2]=r1
      results[i,3]=r2
  return results



In [ ]:
Y=thesis(Original)
#print(Y.shape)
#print(Y)

In [ ]:
impdata=pd.DataFrame({'Itt1_Bias':Y[:,0],'Itt2_Bias':Y[:,1], 'Itt1_RMSE':Y[:,2], 'Itt2_RMSE':Y[:,3] })
#print(impdata.head())

In [ ]:
final=impdata.mean()
print(final.round(4))

Itt1_Bias   -0.0003
Itt2_Bias    0.0008
Itt1_RMSE    0.0176
Itt2_RMSE    0.0367
dtype: float64
